In [1]:
import os, sys
import numpy as np

from pathlib import Path

from astropy.io import fits, ascii
from astropy.table import Table, Column, vstack, hstack, join
from astropy.cosmology import FlatLambdaCDM
# from astropy.constants import c as c_speed
# from astropy import units as u

from scipy import interpolate
import scipy.stats as stats

# import itertools as it

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams.update({"text.usetex": True})

%matplotlib inline

sys.path.append("./py")
from utils import *

sys.path.append("/Users/aberti/Desktop/research")
# from plotutils import get_corners, fig_labels

from params import BASEDIR, DATADIR, SIMDIR, H0, Om0
cosmo = FlatLambdaCDM(H0=H0, Om0=Om0)

# import Corrfunc
# from Corrfunc.theory.DDrppi import DDrppi
# from Corrfunc.mocks.DDrppi_mocks import DDrppi_mocks
# from Corrfunc.theory.wp import wp as wp_corrfunc


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/matplotlib/__init__.py:550: UserWarning: matplotlibrc text.usetex can not be used unless ghostscript-9.00 or later is installed on your system
  % gs_req)


In [50]:
def _sph_to_cart(RA,Dec,dcom):
    theta = 90.0 - Dec
    phi   = RA
    
    x = dcom*np.sin(np.deg2rad(theta))*np.cos(np.deg2rad(phi)).data
    y = dcom*np.sin(np.deg2rad(theta))*np.sin(np.deg2rad(phi)).data
    z = dcom*np.cos(np.deg2rad(theta)).data
    
    return x, y, z



def _cart_to_sph(x, y, z, dcom=0., LOS_err_sig=0., boxsize=250., pimax=150., wide=False):
    rho = np.sqrt( (x + dcom)**2 + y**2 + z**2 )
    print(f"rho min/max/range:\t{np.min(rho):.1f}\t{np.max(rho):.1f}\t{np.max(rho)-np.min(rho):.1f}")
    
    if (LOS_err_sig > 0):
        i = 1
        #-- smear out rho values
        rho_scattered = np.random.normal(rho, LOS_err_sig)
        #-- get x_scattered (in terms of rho_scattered)
        x_scattered = np.sqrt( rho_scattered**2 - (y**2 + z**2) ) - dcom
        
        if wide:
            xmin = -(boxsize/2. + pimax)
            xmax =  (boxsize/2. + pimax)
        else:
            xmin = -boxsize/2.
            xmax =  boxsize/2.
        width = xmax - xmin
        print(f"x_scattered min/max:\t{np.min(x_scattered):.1f}\t{np.max(x_scattered):.1f}")
        print(f"x min/max (allowed):\t{xmin:.1f}\t{xmax:.1f}")
        
        while (np.min(x_scattered) < xmin) | (np.max(x_scattered) > xmax):
            x_scattered[x_scattered > xmax] = x_scattered[x_scattered > xmax] - width
            x_scattered[x_scattered < xmin] = x_scattered[x_scattered < xmin] + width
            print(f"     ...wrapped ({i}):\t{np.min(x_scattered):.1f}\t{np.max(x_scattered):.1f}")
            i += 1

        x   = x_scattered
        rho = np.sqrt( (x + dcom)**2 + y**2 + z**2 )
        
    theta = np.arccos( z / rho )
    phi   = np.arcsin( y / np.sqrt((x + dcom)**2 + y**2) )
    
    cz    = rho
    delta = np.rad2deg(np.pi/2. - theta)
    alpha = np.rad2deg(phi)

    if np.min(alpha) < 0:
        alpha += 180

    print(f"cz min/max/range:\t{np.min(cz):.1f}\t{np.max(cz):.1f}\t{np.max(cz)-np.min(cz):.1f}")
    print(f"alpha min/max/range:\t{np.min(alpha):.4f}\t{np.max(alpha):.4f}\t{np.max(alpha)-np.min(alpha):.1f}")
    print(f"delta min/max/range:\t{np.min(delta):.4f}\t{np.max(delta):.4f}\t{np.max(delta)-np.min(delta):.1f}\n")
    
    return alpha, delta, cz


In [ ]:
cat  = Table(np.load(f"{BASEDIR}/mocks/bolshoip/vpeak/zsnap0p43152_zmaglim20p7_MW1limn22p25_scatter20pct.npy"))
band = "MW1"

abs_mag_bins_clust = get_abs_mag_bins_clust(zmin,band)

col = "mag_bindex"
if col in cat.colnames:
    cat.remove_column(col)
cat.add_column(Column( np.zeros(len(cat), dtype=int), name=col ))

abs_mag = cat[band]

print(" max\t min \tpct \tN_gal")
H, bin_edges = np.histogram(abs_mag, bins=abs_mag_bins_clust)
for i in range(len(bin_edges)-1):
    mask = (abs_mag > bin_edges[i]) & (abs_mag <= bin_edges[i+1])
    cat[col][mask] = i

    print(f"{bin_edges[i]}\t{bin_edges[i+1]}\t{100*H[i]/sum(H):.1f}\t{len(cat[mask]):d}")

print()
for coord in ("x","y","z"):
    print(f"{coord}  {np.min(cat[coord])}  {np.max(cat[coord])}")
    

In [43]:
quiet = False
debug = True

#-- data parameters
d          = "north"
zmin, zmax = 0.4, 0.5
mz_lim     = 20.7

#-- define bins for clustering (data and mocks)
nbins   = 19
rp_min  = 0.0333
rp_max  = 64.4098
rp_bins = np.logspace( np.log10(rp_min), np.log10(rp_max), nbins + 1 )
rp_mids = np.array([ 10**np.mean(np.log10(rp_bins)[i:i+2]) for i in np.arange(len(rp_bins)-1) ])
binfile = f"{BASEDIR}/clustering/rp_bins.txt"

#-- correlation function parameters
autocorr    = 0
cosmology   = 2
nthreads    = 2
boxsize     = 250.
pimax       = 150.
LOS_err_sig = 50
rand_coeff  = 20

#-- housekeeping
ztag      = "{:.2f}-{:.2f}".format(zmin, zmax)
ztag      = ztag.replace(".","p")
zsnap     = zsnaps[int(10*zmin)]
dcom      = cosmo.comoving_distance(zsnap).value
zsnap_tag = str(zsnap).replace(".","p")
zlim_tag  = str(mz_lim).replace(".","p")

#-- halo catalog with model MW1 magnitudes
cat = Table(np.load(f"{BASEDIR}/sims/vpeak_select_zsnap{zsnap_tag}_zmaglim{zlim_tag}.npy"))

#MW1_bins = [-21.2,-21.7,-22.2,-22.7,-23.2,-25.2][::-1]
MW1_bins = None

if MW1_bins != None:
    mag_binning = True
    col = "MW1_bin"
    if col in cat.colnames: cat.remove_column(col)
    cat.add_column(Column( np.zeros(len(cat), dtype=int), name=col ))

    MW1 = cat["MW1_assigned_scattered"]

    if not quiet: print("\n max\t min \tpct \tN_gal")
    H, bin_edges = np.histogram(MW1, bins=MW1_bins)
    for i in range(len(bin_edges)-1):
        mask = (MW1 > bin_edges[i]) & (MW1 <= bin_edges[i+1])
        cat[col][mask] = i
        if not quiet: print(f"{bin_edges[i]}\t{bin_edges[i+1]}\t{100*H[i]/sum(H):.1f}\t{len(cat[mask]):d}")
else:
    mag_binning = False
    
if debug:
    rand_coeff = 5 if mag_binning else 1


In [1]:
pimax = 150.
rand_coeff = 5

zsnap = zsnaps[int(10*zmin)]
dcom  = cosmo.comoving_distance(zsnap).value

LOS_err_sig = 30

#-- stack simulation volume
z_stacked = np.concatenate( [cat["x"], cat["x"], cat["x"]] ) - boxsize/2
y_stacked = np.concatenate( [cat["y"], cat["y"], cat["y"]] ) - boxsize/2
x_stacked = np.concatenate( [np.array(cat["z"])-1.5*boxsize,
                             np.array(cat["z"])-0.5*boxsize,
                             np.array(cat["z"])+0.5*boxsize] )
mag_bindex_stacked = np.concatenate( [cat["mag_bindex"], cat["mag_bindex"], cat["mag_bindex"]] )

#-- wide data mask (within boxsize +/- pimax)
wide_mask = np.abs(x_stacked) < (boxsize/2. + pimax)
z_wide = z_stacked[wide_mask]
y_wide = y_stacked[wide_mask]
x_wide = x_stacked[wide_mask]

mag_bindex_wide = mag_bindex_stacked[wide_mask]

#-- place coordinates at comoving LOS distance corresponding to zsnap;
#-- add gaussian LOS error with sigma=LOS_err_sig
#-- wrap under/overflow coordinates back into wide volume
print("DATA WIDE")
alpha_wide, delta_wide, cz_wide = _cart_to_sph(x_wide, y_wide, z_wide, LOS_err_sig=LOS_err_sig, dcom=dcom, wide=True)

#-- select "narrow" mock data (sample1 for cross-correlation with sample2=wide_mock_data)
#-- narrow data mask (within boxsize only)
narrow_mask_cart = np.abs(x_wide) < boxsize/2.
x_narrow = x_wide[narrow_mask_cart]
y_narrow = y_wide[narrow_mask_cart]
z_narrow = z_wide[narrow_mask_cart]

#-- select sample1 from sample2
w,_,_ = _sph_to_cart(alpha_wide-180, delta_wide, cz_wide)
narrow_mask  = np.abs(w - dcom) < boxsize/2
narrow_mask  = np.abs(w - dcom) < boxsize/2

alpha_narrow = alpha_wide[narrow_mask]
delta_narrow = delta_wide[narrow_mask]
cz_narrow    = cz_wide[narrow_mask]

# print(np.min(alpha_narrow), np.max(alpha_narrow))
# print(np.min(delta_narrow), np.max(delta_narrow))

mag_bindex_narrow = mag_bindex_wide[narrow_mask]

#-- get sizes of sample1 and sample2
ND_narrow = len(alpha_narrow)
ND_wide   = len(alpha_wide)

#-- create randoms for sample1 and sample2
NR_narrow = rand_coeff*ND_narrow
NR_wide   = rand_coeff*ND_wide

#-- narrow randoms
RZ_narrow = np.random.uniform(-boxsize/2, boxsize/2, NR_narrow)
RY_narrow = np.random.uniform(-boxsize/2, boxsize/2, NR_narrow)
RX_narrow = np.random.uniform(-boxsize/2, boxsize/2, NR_narrow)# + dcom

#-- wide randoms
RZ_wide = np.random.uniform(-boxsize/2, boxsize/2, NR_wide)
RY_wide = np.random.uniform(-boxsize/2, boxsize/2, NR_wide)
RX_wide = np.random.uniform(-(boxsize/2 + pimax), boxsize/2 + pimax, NR_wide)# + dcom

#-- convert random coordinates to spherical for clustering measurements
print("RANDOM NARROW")
alphaR_narrow, deltaR_narrow, czR_narrow = _cart_to_sph(RX_narrow, RY_narrow, RZ_narrow, LOS_err_sig=LOS_err_sig, dcom=dcom)
print("RANDOM WIDE")
alphaR_wide, deltaR_wide, czR_wide = _cart_to_sph(RX_wide, RY_wide, RZ_wide, LOS_err_sig=LOS_err_sig, dcom=dcom, wide=True)



In [ ]:
%%time

autocorr  = 0
cosmology = 2
nthreads  = 2

# DS = 10
# D1 = (alpha1[::DS], delta1[::DS], cz1[::DS])
# D2 = (alpha2[::DS], delta2[::DS], cz2[::DS])
# R1 = (alpha1R[::DS], delta1R[::DS], cz1R[::DS])
# R2 = (alpha2R[::DS], delta2R[::DS], cz2R[::DS])

for idx in np.unique(cat["mag_bindex"])[1:2]:
    print("\n========================================")
    print(f"Mag bin {idx}...")
#     mask_wide   = mag_bindex_wide == idx 
#     mask_narrow = mag_bindex_narrow == idx
    mask_wide   = np.ones(len(mag_bindex_wide), dtype=bool) 
    mask_narrow = np.ones(len(mag_bindex_narrow), dtype=bool)

    D_narrow = (alpha_narrow[mask_narrow], delta_narrow[mask_narrow], cz_narrow[mask_narrow])
    D_wide   = (alpha_wide[mask_wide], delta_wide[mask_wide], cz_wide[mask_wide])

    ND_narrow = len(D_narrow[0])
    ND_wide   = len(D_wide[0])

    print(ND_narrow, ND_wide)
    for coord,tag in zip(D_narrow,(" alpha_narrow"," delta_narrow","    cz_narrow")):
        print(f"{tag} min / max: {np.min(coord):.1f}\t{np.max(coord):.1f}")
    print()
    for coord,tag in zip(D_wide,(" alpha_wide  "," delta_wide  ","    cz_wide  ")):
        print(f"{tag} min / max: {np.min(coord):.1f}\t{np.max(coord):.1f}")
    print()

    NR_narrow = rand_coeff*ND_narrow
    NR_wide   = rand_coeff*ND_wide
        
    R_narrow = (alphaR_narrow[:NR_narrow], deltaR_narrow[:NR_narrow], czR_narrow[:NR_narrow])
    R_wide   = (alphaR_wide[:NR_wide], deltaR_wide[:NR_wide], czR_wide[:NR_wide])
        
    for coord,tag in zip(R_narrow,("alphaR_narrow","deltaR_narrow","   czR_narrow")):
        print(f"{tag} min / max: {np.min(coord):.1f}\t{np.max(coord):.1f}")
    print()
    for coord,tag in zip(R_wide,("  alphaR_wide","  deltaR_wide","     czR_wide")):
        print(f"{tag} min / max: {np.min(coord):.1f}\t{np.max(coord):.1f}")
        
#     pairs_D1D2 = [ D_narrow, D_wide ]
#     pairs_D1R2 = [ D_narrow, R_wide ]
#     pairs_D2R1 = [ D_wide, R_narrow ]
#     pairs_R1R2 = [ R_narrow, R_wide ]

#     pair_sets   = [pairs_D1D2, pairs_D1R2, pairs_D2R1, pairs_R1R2]
    pair_sets = [[D_narrow,D_wide], [D_narrow,R_wide], [D_wide,R_narrow], [R_narrow,R_wide]]
    pair_counts = []

    for pair in pair_sets:
        alpha1, delta1, cz1 = pair[0]
        alpha2, delta2, cz2 = pair[1]

        print(f"\nPair {len(pair_counts)+1}")
        print(f"alpha1 min / max: {np.min(alpha1):.1f}\t{np.max(alpha1):.1f}")
        print(f"alpha2 min / max: {np.min(alpha2):.1f}\t{np.max(alpha2):.1f}")
        
        kwargs = dict(RA2=alpha2, DEC2=delta2, CZ2=cz2, weights2=None, is_comoving_dist=True, verbose=False)
        counts = DDrppi_mocks(autocorr, cosmology, nthreads, pimax, rp_bins, alpha1, delta1, cz1, **kwargs)

        pair_counts.append(counts)
        #print(f"Pair {len(pair_counts)} done")

    #-- sets of pair counts (DD, DR, RD, RR)
    count_sets = [ pair_counts[pair_idx]["npairs"] for pair_idx in range(len(pair_sets)) ]

    D1D2, D1R2, D2R1, R1R2 = count_sets
    ND1, ND2, NR1, NR2 = ND_narrow, ND_wide, NR_narrow, NR_wide
    
    xi = ( D1D2/(ND1*ND2) - D1R2/(ND1*NR2) - D2R1/(ND2*NR1) ) / ( R1R2/(NR1*NR2) ) + 1

    #-- get wp for multiple values of pimax
    pimax_array = np.arange(10.,151.,10.)

    wp_from_DDrppi = {}
    wp_from_DDrppi["rp_cen"] = rp_mids

    for this_pimax in pimax_array:
        this_wp = 2*sum([ xi[int(i)::int(pimax)] for i in range(int(pimax)) ][:int(this_pimax)])
        wp_from_DDrppi[str(this_pimax)] = this_wp

    result = Table( wp_from_DDrppi )

result

# zsnap_tag = str(zsnap).replace(".","p")
# np.save(f"{BASEDIR}/clustering/wp_zsnap{zsnap_tag}_{d}_zmaglim{zmag_tag}.npy", result)


In [ ]:
fig, ax = plt.subplots(figsize=(16,8))

ax.scatter(x_wide[~narrow_mask_cart][::2], y_wide[~narrow_mask_cart][::2], s=0.5, color="gray")
ax.scatter(x_wide[narrow_mask_cart][::2], y_wide[narrow_mask_cart][::2], s=0.5, color="gray")
ax.scatter(x_narrow[::2], y_narrow[::2], s=0.5, color="red")

ax.set_xlim(np.min(x_wide)-10, np.max(x_wide)+10)
ax.set_ylim(np.min(y_wide)-10, np.max(y_wide)+10)

ax.set_xlabel(r"${\rm LOS\ distance}\ (h^{-1}{\rm Mpc})$", fontsize=24)
ax.set_ylabel(r"$y\ (h^{-1}{\rm Mpc})$", fontsize=24)

ax.set_title(r"${\rm Extended\ simulation\ volume\ in\ cartesian\ coordinates\ without\ LOS\ error}\ (z_{\rm sim}=\ $"+f"${zsnap})$", fontsize=24)

zsim = str(zsnap).replace(".","p")
#plt.savefig(f"{BASEDIR}/figures/zsim{zsim}_LOS-y_no_error.png", bbox_inches="tight", pad_inches=0.1)

plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(15,8))

ax.scatter(cz_wide[~narrow_mask][::2], alpha_wide[~narrow_mask][::2]-180, s=0.5, color="gray")
ax.scatter(cz_wide[narrow_mask][::2], alpha_wide[narrow_mask][::2]-180, s=0.5, color="gray")
ax.scatter(cz_narrow[::2], alpha_narrow[::2]-180, s=0.5, color="red", alpha=0.5)

ax.set_xlabel(r"${\rm LOS\ distance}\ (h^{-1}{\rm Mpc})$", fontsize=24)
ax.set_ylabel(r"$\alpha\ (\deg)$", fontsize=24)

ax.set_xlim(np.min(cz2)-10, np.max(cz2)+10)
ax.set_ylim(-5.5,5.5)
ax.set_yticks(np.arange(-5,6,1))

ax.set_title(r"${\rm Extended\ simulation\ volume\ in}\ (\alpha,\delta,d_{\rm com})\ {\rm with\ LOS\ error}\ (z_{\rm sim}=\ $"+f"${zsnap})$", fontsize=24)

#zsim = str(zsnap).replace(".","p")
plt.savefig(f"{BASEDIR}/figures/zsim{zsim}_alpha-cz_with_error.png", bbox_inches="tight", pad_inches=0.1)

plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(16,8))

ax.scatter(czR_wide[::2*rand_coeff], deltaR_wide[::2*rand_coeff], s=0.5, color="gray")
ax.scatter(czR_narrow[::2*rand_coeff], deltaR_narrow[::2*rand_coeff], s=0.5, color="red", alpha=0.5)

ax.set_xlabel(r"${\rm LOS\ distance}\ (h^{-1}{\rm Mpc})$", fontsize=24)
ax.set_ylabel(r"$\delta\ (\deg)$", fontsize=24)

ax.set_xlim(np.min(czR_wide)-10, np.max(czR_wide)+10)
ax.set_ylim(-5.5,5.5)
ax.set_yticks(np.arange(-5,6,1))

ax.set_title(r"${\rm Randoms\ for\ extended\ simulation\ volume\ in}\ (\alpha,\delta,d_{\rm com})\ (z_{\rm sim}=\ $"+f"${zsnap})$", fontsize=24)
plt.savefig(f"{BASEDIR}/figures/zsim{zsim}_delta-cz_randoms.png", bbox_inches="tight", pad_inches=0.1)

plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

ax.scatter(alpha_wide[::2]-180, delta_wide[::2], s=1, color="gray")
ax.scatter(alpha_narrow[::2]-180, delta_narrow[::2], s=1, alpha=0.3, color="red")

ax.set_xlabel(r"$\alpha\ (\deg)$", fontsize=24)
ax.set_ylabel(r"$\delta\ (\deg)$", fontsize=24)

ax.set_xlim(-5.2,5.2)
ax.set_ylim(-5.2,5.2)

ax.set_title(r"${\rm Along\ LOS\ of\ extended\ simulation\ volume\ (distorted)}$", fontsize=24)

plt.savefig(f"{BASEDIR}/figures/zsim{zsim}_alpha-delta_with_error.png", bbox_inches="tight", pad_inches=0.1)

plt.show()


In [ ]:
w1, u1, v1 = _sph_to_cart(alpha_narrow-180, delta_narrow, cz_narrow)
w2, u2, v2 = _sph_to_cart(alpha_wide-180, delta_wide, cz_wide)

fig, ax = plt.subplots(figsize=(16,8))

ax.scatter(w2[~narrow_mask][::2], v2[~narrow_mask][::2], s=0.5, color="gray")
ax.scatter(w2[narrow_mask][::2], v2[narrow_mask][::2], s=0.5, color="gray")
ax.scatter(w1[::2], v1[::2], s=0.5, color="red", alpha=0.5)

ax.set_xlim(np.min(w2)-10, np.max(w2)+10)
ax.set_ylim(np.min(v2)-10, np.max(v2)+10)

ax.set_xlabel(r"${\rm LOS\ distance}\ (h^{-1}{\rm Mpc})$", fontsize=24)
ax.set_ylabel(r"$y\ (h^{-1}{\rm Mpc})$", fontsize=24)

ax.set_title(r"${\rm Extended\ simulation\ volume\ in\ cartesian\ coordinates\ with\ LOS\ error}\ (z_{\rm sim}=\ $"+f"${zsnap})$", fontsize=24)

plt.savefig(f"{BASEDIR}/figures/zsim{zsim}_LOS-y_with_error.png", bbox_inches="tight", pad_inches=0.1)

plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

ax.scatter(u2, v2, s=0.5, color="gray")
ax.scatter(u1, v1, s=0.5, alpha=0.25, color="red")

ax.set_xlabel(r"$x\ (h^{-1}{\rm Mpc})$", fontsize=24)
ax.set_ylabel(r"$y\ (h^{-1}{\rm Mpc})$", fontsize=24)

ax.set_xlim(-130,130)
ax.set_ylim(-130,130)

ax.set_title(r"${\rm Along\ LOS\ of\ extended\ simulation\ volume\ (cartesian)}$", fontsize=24)
plt.savefig(f"{BASEDIR}/figures/zsim{zsim}_x-y_with_error.png", bbox_inches="tight", pad_inches=0.1)

plt.show()
